# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
                   WITH REPLICATION =
                   {'class': 'SimpleStrategy', 'replication_factor': 1}""")

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifydb')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# pandas dataframe for later sanity checks
df = pd.read_csv('event_datafile_new.csv')

In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = 'CREATE TABLE IF NOT EXISTS session_history'
query += '(artist text, song_title text, song_length double, session_id int, item_in_session int,\
           PRIMARY KEY(session_id, item_in_session))'
session.execute(query)

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_history (artist, song_title, song_length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute('SELECT * FROM session_history WHERE session_id=338 AND item_in_session=4')
for row in rows: print(row)

Row(session_id=338, item_in_session=4, artist='Faithless', song_length=495.3073, song_title='Music Matters (Mark Knight Dub)')


In [12]:
# sanity check
df.loc[(df.sessionId == 338) & (df.itemInSession == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5810,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = 'CREATE TABLE IF NOT EXISTS user_history '
query += '(artist text, song_title text, first_name text, last_name text, user_id int, session_id int,\
           item_in_session int, PRIMARY KEY(user_id, session_id, item_in_session))'
session.execute(query)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (artist, song_title, first_name, last_name, user_id,\
                                           session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, 
                        (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [15]:
rows = session.execute('SELECT * FROM user_history WHERE user_id=10 AND session_id=182')
for row in rows: print(row)

Row(user_id=10, session_id=182, item_in_session=0, artist='Down To The Bone', first_name='Sylvie', last_name='Cruz', song_title="Keep On Keepin' On")
Row(user_id=10, session_id=182, item_in_session=1, artist='Three Drives', first_name='Sylvie', last_name='Cruz', song_title='Greece 2000')
Row(user_id=10, session_id=182, item_in_session=2, artist='Sebastien Tellier', first_name='Sylvie', last_name='Cruz', song_title='Kilometer')
Row(user_id=10, session_id=182, item_in_session=3, artist='Lonnie Gordon', first_name='Sylvie', last_name='Cruz', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [16]:
# sanity check
df.loc[(df.userId == 10) & (df.sessionId == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6805,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
6806,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
6807,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
6808,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [17]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = 'CREATE TABLE IF NOT EXISTS song_history '
query += '(song_title text, first_name text, last_name text, user_id int, PRIMARY KEY(song_title, user_id))'
session.execute(query)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (song_title, first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [19]:
rows = session.execute("SELECT first_name, last_name FROM song_history WHERE song_title='All Hands Against His Own'")
for row in rows: print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


In [20]:
df[['song', 'firstName', 'lastName']].drop_duplicates().loc[df.song == 'All Hands Against His Own']

,song,firstName,lastName
1313,All Hands Against His Own,Jacqueline,Lynch
1839,All Hands Against His Own,Tegan,Levine
5282,All Hands Against His Own,Sara,Johnson


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions

In [22]:
for table in ['session_history', 'user_history', 'song_history']:
    session.execute(f'DROP TABLE IF EXISTS {table}')

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()